# Proyecto 1: Detección de ironía.

In [1]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

/home/david/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Lectura de archivos

In [2]:
ironicos = pd.read_csv('ironicos_reduced.txt', header = None, delimiter='\t', names=["Tuits"])
no_ironicos = pd.read_csv('no_ironicos_reduced.txt', header = None, delimiter='\t', names=["Tuits"])

ironicos['Clasificacion'] = "Ironico"
no_ironicos['Clasificacion'] = "No_Ironico"

#Se juntan los tuits ironicos y no ironicos en un solo dataset
data = ironicos.append(no_ironicos)
data = data.reset_index(drop=True)
data = data.reindex(np.random.permutation(data.index))
data = data.reset_index(drop=True)
data

,Tuits,Clasificacion
0,"c.c: te conozco hace mil y bueno nada, me acue...",No_Ironico
1,Contrapunto #felizviernes #ironía #harta http:...,Ironico
2,"si no avisan que hace frio, no me doy cuenta",Ironico
3,"@Fergiron @DaniPintoB @juanpe_ab claro, es que...",Ironico
4,"Aquí tienen mi unfollow, sigo siendo azul, así...",No_Ironico
5,Hay gente que no se merecen ni la 1/4 parte de...,No_Ironico
6,Pero volví\nPero mi tl esta she muerta,No_Ironico
7,"Porque como tú no ahí nadie más, porque como t...",No_Ironico
8,"Mis mañanas se basa en ver LQSA, a la 1 los si...",Ironico
9,Adoro la puta canción 😍 P.D: Yo voy para la vo...,Ironico


# Conversión de filas a vectores


In [3]:
def tuit_Vector(mensaje):
    mensaje = mensaje.split()
    for i in range(len(mensaje)):        
        mensaje[i] = mensaje[i].replace(".","")        
        mensaje[i] = mensaje[i].replace("?","")
        mensaje[i] = mensaje[i].replace("¿","")
        mensaje[i] = mensaje[i].replace("!","")
        mensaje[i] = mensaje[i].replace(",","") 
        mensaje[i] = mensaje[i].replace('"',"") 
        mensaje[i] = mensaje[i].replace(":","")
        mensaje[i] = mensaje[i].replace(";","") 
        if(mensaje[i].startswith('#')):                        
            mensaje[i] = mensaje[i].replace(mensaje[i],"")                       
        mensaje[i] = mensaje[i].lower()   
    return mensaje

#Se eliminan simbolos que no son necesarios
print (data.Tuits.head().apply(tuit_Vector))


#numero de plabras unicas en el conjunto de datos
transformar = CountVectorizer(analyzer=tuit_Vector).fit(data['Tuits'])

#Numero de palabras totales
print (len(transformar.vocabulary_))

0    [cc, te, conozco, hace, mil, y, bueno, nada, m...
1                      [contrapunto, , , , http\\link]
2    [si, no, avisan, que, hace, frio, no, me, doy,...
3    [@fergiron, @danipintob, @juanpe_ab, claro, es...
4    [aquí, tienen, mi, unfollow, sigo, siendo, azu...
Name: Tuits, dtype: object
14302


In [4]:
#Proponemos un ejemplo para analizar
tuit1 = data['Tuits'][0]
print(tuit1)


#ver la ocurrencia de las palabras en un tuit, y ver cuantas
#veces se repite esa palabra en dicho tuit
twt1 = transformar.transform([tuit1])
print (twt1)

#Ver las palabras por los indices mostrados en el paso anterior
print (transformar.get_feature_names()[8614])
print (transformar.get_feature_names()[0])  #Es el espacio en blanco
print (transformar.get_feature_names()[3828])

#Se muestra el id de la palabra "me"
print (transformar.vocabulary_.get("me"))

#Se crean las matrices de palabras ya transformadas
matriz = transformar.fit_transform(data['Tuits'])

print(matriz.shape)


c.c: te conozco hace mil y bueno nada, me acuerdo cuando eramos \"novios\" en primaria jajajajaj, nqv, te re mil quiero horrible
  (0, 1303)	1
  (0, 1554)	1
  (0, 2838)	1
  (0, 3284)	1
  (0, 3878)	1
  (0, 4176)	1
  (0, 5340)	1
  (0, 5567)	1
  (0, 6820)	1
  (0, 7059)	1
  (0, 7594)	1
  (0, 8617)	1
  (0, 8801)	2
  (0, 9147)	1
  (0, 9379)	1
  (0, 10614)	1
  (0, 10936)	1
  (0, 11045)	1
  (0, 12438)	2
  (0, 13642)	1
mcdinero

confiadaaaa
8617
(5838, 14302)


# Ver las palabras por frecuencia de ocurrencia del termino en los mensajes

In [5]:
#se utiliza la funcion TF-IDF la cual sirve para reflejar la importancia
#de cada palabra en el tuit
tfidf_Trans = TfidfTransformer().fit(matriz)


#Se muestra un ejemplo para el tuit con el que se trabajo antes
#Se muestra cada palabra por indices y su importancia
tfidf1 = tfidf_Trans.transform(twt1)
print (tfidf1)

#Se puede ver el valor de importancia de cada palabra en nuestro conjunto de datos
print (tfidf_Trans.idf_[transformar.vocabulary_['me']])
print (tfidf_Trans.idf_[transformar.vocabulary_['desigualdad']])

#Se transforma toda la matriz de una vez con los datos actualizados
matriz_tfidf = tfidf_Trans.transform(matriz)


  (0, 13642)	0.080696182194
  (0, 12438)	0.225770759509
  (0, 11045)	0.180426910908
  (0, 10936)	0.164902776283
  (0, 10614)	0.275652529942
  (0, 9379)	0.288688611184
  (0, 9147)	0.14642566667
  (0, 8801)	0.436091585084
  (0, 8617)	0.0833986583169
  (0, 7594)	0.244117955027
  (0, 7059)	0.240331120621
  (0, 6820)	0.156784802089
  (0, 5567)	0.248411113791
  (0, 5340)	0.0867981476977
  (0, 4176)	0.107182230155
  (0, 3878)	0.240331120621
  (0, 3284)	0.266403283105
  (0, 2838)	0.162913424366
  (0, 1554)	0.208796545706
  (0, 1303)	0.288688611184
2.59397324873
8.97916764772


# Entrenamiento de datos  y pruebas

In [6]:
#Se separan los datos para el entrenamiento y pruebas
objetivo = data["Clasificacion"]

#Se separa el conjiunto de datos en subconjuntos de entrenamiento y pruebas
matriz_tfidf_train, matriz_tfidf_test, objetivo_train, objetivo_test = train_test_split(matriz_tfidf,
                                                                                             objetivo, test_size = 0.33, 
                                                                                             random_state = 10)
#Se entrena el modelo
clf = svm.SVC(kernel="linear")
clf.fit(matriz_tfidf_train, objetivo_train)

#Evaluacion
predicted = clf.predict(matriz_tfidf_test)

#Se muestra la precision del modelo y la matriz de confusion
print ('Precision: ', accuracy_score(objetivo_test, predicted, normalize = True))
print ('Matriz de Confusion\n', confusion_matrix(objetivo_test, predicted))
print ('Filas: Valor Real\nColumnas: Valor Predicho')
print ("\n")
print("Reporte: ")
print(classification_report(objetivo_test, predicted))

Precision:  0.960560456668
Matriz de Confusion
 [[890  53]
 [ 23 961]]
Filas: Valor Real
Columnas: Valor Predicho


Reporte: 
             precision    recall  f1-score   support

    Ironico       0.97      0.94      0.96       943
 No_Ironico       0.95      0.98      0.96       984

avg / total       0.96      0.96      0.96      1927

